# Huffman Coding

In [2]:
import heapq
class BinaryTreeNode:
    def __init__(self, value, freq):
        self.value = value
        self.freq = freq
        self.left = None
        self.right = None
    
    def __lt__(self, other):
        return self.freq < other.freq
    
    def __eq__(self, other):
        return self.freq == other.freq
        
class HuffmanCoding:
    def __init__(self,path):
        self.path = path
        self.__heap = []
        self.__codes = {}
        
    def __make_freq_dict(self, text):
        freq_dict = {}
        for char in text:
            freq_dict[char] = freq_dict.get(char, 0) + 1
        return freq_dict
    
    def __buildHeap(self, freq_dict):
        for key in freq_dict:
            freq = freq_dict[key]
            binaryTreeNode = BinaryTreeNode(key, freq)
            heapq.heappush(self.__heap, binaryTreeNode)
            
    def __makeBinaryTree(self):
        while len(self.__heap > 1):
            min1 = heapq.heappop(self.__heap)
            min2 = heapq.heappop(self.__heap)
            freq_sum = min1.freq + min2.freq
            
            binTreeNode = BinaryTreeNode(None , freq_sum)
            binTreeNode.left = min1
            binTreeNode.right = min2
            heapq.heappush(self.__heap, binTreeNode)
        return
    
    
    def __constructCodesHelp(self, root, curr_bits):
        if root is None:
            return
        if root.value is not None:
            self.__codes[root.value] = curr_bits
            return
        self.__constructCodesHelp(root.left, curr_bits + "0")
        self.__constructCodesHelp(root.right, curr_bits + "0")
    
    def __constructCodes(self):
        root = heapq.heappop(self.__heap)
        self.__constructCodesHelp(root, "")
        
    def __encodeText(self, text):
        encoded_text = ""
        for char in text:
            encoded_text += self.__codes[char]
        return encoded_text
    
        
        
        
    
    def compress(self):
        # get file from path 
        with open(self.path, r) as f:
            text = f.read()
        # read text from file
        
        # make frequency dictionay using the text
        freq_dict = self.__make_freq_dict(text)
        
        
        # construct the heap from the freq dictionary
        self.__buildHeap(freq_dict)
        
        # construct the binary tree from the heap
        self.__makeBinaryTree()
        
        
        # Construct the codes from the binary tree
        self.__constructCodes()
        
        # Create the encoded text
        
        encoded_text = self.__encodeText(text)
        
        # Put the encoded text into the binary file 
        
        
        # return the binary file as output
        
        pass
    
    def decompress(self):
        pass

In [8]:
t = "abcabcdef"
f = {}
for i in t:
    f[i] = f.get(i, 0) + 1

In [9]:
f

{'a': 2, 'b': 2, 'c': 2, 'd': 1, 'e': 1, 'f': 1}